In [ ]:
# Install Pennylane
!pip install pennylane

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import pennylane as qml

# Load the brain stroke dataset
data = pd.read_csv('/content/brain_stroke.csv')

# Encode the 'smoking_status' column
encoder = LabelEncoder()
data['smoking_status'] = encoder.fit_transform(data['smoking_status'])

# Drop the 'gender', 'ever_married', 'work_type', and 'Residence_type' columns
data = data.drop(['gender', 'ever_married', 'work_type', 'Residence_type'], axis=1)

# Split data into features (X) and target (y)
X = data.drop('stroke', axis=1)
y = data['stroke']

# Scale the features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the quantum device
dev = qml.device("default.qubit", wires=X.shape[1])

# Quantum circuit definition
@qml.qnode(dev)
def qnn_circuit(weights, x):
    qml.RY(np.sum(np.broadcast_to(weights, x.shape) * x), wires=0)
    return qml.expval(qml.PauliZ(0))

# Quantum neural network
def qnn_model(weights, X):
    return np.array([qnn_circuit(weights, x) for x in X])

# Loss function
def mean_squared_loss(weights, X, y):
    predictions = qnn_model(weights, X)
    return np.mean((predictions - y)**2)

# Initialize random weights
np.random.seed(0)
num_weights = X.shape[1]
init_weights = np.random.random(num_weights)

# Optimize the QCNN
opt = qml.GradientDescentOptimizer(0.1)
weights = init_weights.copy()

for i in range(100):
    weights = opt.step(lambda w: mean_squared_loss(w, X_train, y_train), weights)

# Make predictions
predictions = qnn_model(weights, X_test)

# Evaluate the performance
test_rmse = np.sqrt(mean_squared_error(y_test, predictions))
print('QCNN Test RMSE:', test_rmse)

# Quantum Transfer Learning (QTL)
# Define a new quantum neural network for transfer learning
def qnn_model_qtl(weights, X):
    return np.array([qnn_circuit(weights, x) for x in X])

# Loss function for QTL
def mean_squared_loss_qtl(weights, X, y):
    predictions = qnn_model_qtl(weights, X)
    return np.mean((predictions - y)**2)

# Initialize weights with pre-trained QCNN weights
weights_qtl = weights.copy()

# Optimize the QNN for QTL
for i in range(100):
    weights_qtl = opt.step(lambda w: mean_squared_loss_qtl(w, X_train, y_train), weights_qtl)

# Make predictions using QTL
predictions_qtl = qnn_model_qtl(weights_qtl, X_test)

# Evaluate the performance of QTL
test_rmse_qtl = np.sqrt(mean_squared_error(y_test, predictions_qtl))
print('QTL Test RMSE:', test_rmse_qtl)

/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:157: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


QCNN Test RMSE: 0.6808072620728469


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:157: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


QTL Test RMSE: 0.6808072620728469
